### 1. 프림알고리즘 혼자 구현해보기

In [14]:
from heapq import *
from collections import defaultdict

def prims(start_node, edges):
    adjacent_edges = defaultdict(list)
    
    for weight, n1, n2 in edges:
        adjacent_edges[n1].append((weight, n1,n2))
        adjacent_edges[n2].append((weight , n2, n1))
        
    connected_nodes = set(start_node)
    candidated_edge_list = adjacent_edges[start_node]
    heapify(candidated_edge_list)
    mst = list()
    
    while candidated_edge_list:
        weight, n1, n2 = heappop(candidated_edge_list)
        if n2 not in connected_nodes:
            connected_nodes.add(n2)
            mst.append((weight, n1, n2))
            
            for edge in adjacent_edges[n2]:
                if edge[2] not in connected_nodes:
                    heappush(candidated_edge_list, edge)
                    
    return mst

In [15]:
edges = [
    (7,'A','B'),
    (5,'A','D'),
    (9,'B','D'),
    (8,'B','C'),
    (7,'B','E'),
    (5,'C','E'),
    (7,'D','E'),
    (6,'D','F'),
    (8,'E','F'),
    (9,'E','G'),
    (11,'F','G')
]

In [16]:
prims('A',edges)

[(5, 'A', 'D'),
 (6, 'D', 'F'),
 (7, 'A', 'B'),
 (7, 'B', 'E'),
 (5, 'E', 'C'),
 (9, 'E', 'G')]

### 2. 개선된 프림알고리즘

- 이전에 구현한 프림알고리즘의 경우, 간선을 기준으로 잡으면서 O(ElogE)의 시간복잡도
    - 보통은 노드보다 간선이 많은경우가 많음
    - 따라서 노드를기준으로 한 코드를 작성하여 개선을 하게됨 O(ElogV)
    
- 노드를 기준으로 우선순위 큐 적용 O(ElogV)
    - **초기화 - 정점: key구조 만들고 특정정점의 key값은 0, 나머지 정점들의 key값은 무한대로 놓음**
        모든 정점:: key값은 우선순위 큐에 넣음
    - **가장 key값이 적은 정점: key를 추출한 후**
        - (pop 하므로 해당정점: key 정보는 우선순위 큐에서 삭제됨)
        - (extract min 로직이라고 불림)
    - **해당 정점의 인접한 정점들에 대해 key값과 연결된 가중치 값을 비교하여 key값보다 값이 작으면 해당정점: key값을 갱신(연결된 가중치 값으로)**
        - key값 갱신시 어느노드와 간선으로 부터 들어온 것인지 저장(pi)
        - 정점: key값 갱신시, 우선순위 큐는 최소 key값을 가지는 정점: key를 루트노드로 올려놓도록 재구성함
        - (decrease key 로직이라고 불림)
        
- 개선된 프림알고리즘 구현시 고려사항
    - 우선순위 큐(최소힙) 구조에서, 이미 들어가 있는 데이터 값 변경시 최소값을 가지는 데이터를 루트노드로 올려놓도록 재구성하는 기능 필요
    - 구현 복잡도 줄이기 위해 heapdict 라이브러리 사용

In [33]:
from heapdict import heapdict

def prim(graph, start):
    mst, keys, pi, total_weight = list(), heapdict(), dict(), 0
    for node in graph.keys():
        keys[node] = float('inf')
        pi[node] = None
    keys[start], pi[start] = 0, start
    
    while keys:
        current_node, current_key = keys.popitem()
        mst.append([pi[current_node], current_node, current_key]) # 선택된 최소 간선
        total_weight +=  current_key
        for adjacent, weight in mygraph[current_node].items():
            if adjacent in keys and weight  <keys[adjacent]:
                keys[adjacent] =weight 
                pi[adjacent] = current_node
    return mst, total_weight 

In [37]:
mygraph = {
    'A':{'B':7, 'D':5},
    'B':{'A':7,'D':9,'C':8,'E':7},
    'C':{'B':8, 'E':5},
    'D':{'A':5,'B':9,'F':6,'E':7},
    'E':{'B':7,'D':7,'C':5,'F':8,'G':9},
    'F':{'D':6,'G':11,'E':8},
    'G':{'F':11,'E':9},
}

mst, total_weight = prim(mygraph,'A')
print('MST: ', mst)
print('Total weight: ', total_weight)

MST:  [['A', 'A', 0], ['A', 'D', 5], ['D', 'F', 6], ['D', 'E', 7], ['E', 'C', 5], ['A', 'B', 7], ['E', 'G', 9]]
Total weight:  39


### 3. 시간복잡도
1. keys.popitem() 을 노드의 갯수만큼 반복하기 때문에 O(VlogV)
    1. while구문은 V(노드 갯수)만큼 반복 
    2. heap에서 최소 key값 가지는 노드 정보 pop의 시간복잡도 O(logV)
2. 최초 key생성 시간복잡도:  O(V)
3. 노드의 key값을 업데이트 할때마다, heap이 재구성되야함.O(ElogV)
    
정리
1. O(V+VlogV + ElogV)
    - V는 다른 합하는 값들에 비해 작은값 (무시)
    - (V+E)logV
    - E>V이고, 최대 E == V^2, 따라서 O(ElogV)로 나타낼수있음